In [1]:
#    Copyright 2020 Arkadip Bhattacharya

#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at

#        http://www.apache.org/licenses/LICENSE-2.0

#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.

In [1]:
import os
import bz2
import gzip
import requests
import numpy as np
import netCDF4 as nc
from collections import OrderedDict as OD

#### Fetching the Location of a place in `Berlin, Germany`

- Longitude of the location is - 13.3946
- Latitute of the location is - 52.5246

In [2]:
def create_dir(dir_name):
    try:
        os.mkdir(dir_name)
        print("Dir Created:", dir_name)
    except FileExistsError:
        print( "Dir Already exists:", dir_name)
    return

def fetch_decompress(url_param, file_name):
    if url_param == 'air_temperature_mean':
        ending = 'bz2'
    else:
        ending = 'gz'
    url = 'https://opendata.dwd.de/climate_environment/CDC/grids_germany/hourly/Project_TRY/{}/{}.{}'.format(url_param,file_name, ending)
    with open('./{}/{}'.format(url_param, file_name), 'wb+') as f:
        print('Fetching URL: ', url)
        payload = requests.get(url)
        print("Fetched", file_name)
        if url_param == 'air_temperature_mean':
            dec_payload = bz2.decompress(payload.content)
        else:
            dec_payload = gzip.decompress(payload.content)
        print("Decompressed", file_name)
        f.write(dec_payload)
        print('Completed Writing', file_name)
    return

def delete_file(url_param, file_name):
    name = "./{}/{}".format(url_param, file_name)
    
    if os.path.exists(name):
        os.remove(name)
        print("Deleted "+ name)
    else:
        print("The file " + name + " does not exist")
    return

In [3]:
def getclosest_ij(lats,lons,latpt,lonpt):
    # find squared distance of every point on grid
    dist_sq = (lats-latpt)**2 + (lons-lonpt)**2  
    # 1D index of minimum dist_sq element
    minindex_flattened = dist_sq.argmin()    
    # Get 2D index for latvals and lonvals arrays from 1D index
    return np.unravel_index(minindex_flattened, lats.shape)

In [4]:
def get_value(url_param, file_name, latpt, lonpt):
    output_params = {'air_temperature_mean' : 'temperature',
                     'pressure' : 'SLP',
                     'humidity' : 'humidity',
                     'vapor_pressure' : 'vapor',
                     'wind_direction' : 'DD',
                     'wind_speed' : 'FF'}
    output_variable = output_params[url_param]
    
    with nc.Dataset('./{}/{}'.format(url_param, file_name)) as ncfile:
        print("Opened netCDF: {}".format(file_name))
        lon = ncfile.variables['lon']
        lat = ncfile.variables['lat']
        time = ncfile.variables['time']
        temp = ncfile.variables[output_variable]
        y, x = getclosest_ij(lat[:], lon[:], latpt, lonpt)
        values = []
        times = []
        for day in range(ncfile.dimensions['time'].size):
            times.append(str(nc.num2date(time[day], time.units)))
            values.append(str(temp[day, y, x]))
        return (times, values)

In [5]:
params = OD({'air_temperature_mean' : 'TT',
'pressure' : 'PRED',
'humidity' : 'RH',
'vapor_pressure' : 'X',
'wind_direction' : 'DD',
'wind_speed' : 'FF'})

In [6]:
lon, lat = 13.3946, 52.5246

In [8]:
start_year = 1995
end_year = 1995
start_month = 1
end_month = 12

FNAME = './dataset-hourly-new.csv'
#  Writing Headers
with open(FNAME, 'w+') as csvfile:
    csvfile.write('time,')
    csvfile.write(','.join([i for i in params]))
    csvfile.write('\n')

for year in range(start_year, end_year+1):
    for month in range(start_month, end_month+1):
        values_list = []
        times_list = []
        for url_param, measurement_param in params.items():
            print("Fetching {} of Year: {} of Month: {}".format(url_param, year, month))
            file_name = '{}_{}{:02d}.nc'.format(measurement_param, year, month)
            create_dir(url_param)
            fetch_decompress(url_param, file_name)
            times, values = get_value(url_param, file_name, lat, lon)
            values_list.append(values)
            times_list.append(times)
            
        with open(FNAME, 'a+') as csvfile:
            for day in range(len(values_list[0])):
                final = []
                final.append(times_list[0][day])
                final.append(values_list[0][day])
                final.append(values_list[1][day])
                final.append(values_list[2][day])
                final.append(values_list[3][day])
                final.append(values_list[4][day])
                final.append(values_list[5][day])
                csv_row = ','.join(final)
                csvfile.write(csv_row+'\n')
        for url_param, measurement_param in params.items():
            file_name = '{}_{}{:02d}.nc'.format(measurement_param, year, month)
            delete_file(url_param, file_name)

Fetching air_temperature_mean of Year: 1995 of Month: 1
Dir Already exists: air_temperature_mean
Fetching URL:  https://opendata.dwd.de/climate_environment/CDC/grids_germany/hourly/Project_TRY/air_temperature_mean/TT_199501.nc.bz2
Fetched TT_199501.nc
Decompressed TT_199501.nc
Completed Writing TT_199501.nc
Opened netCDF: TT_199501.nc
Fetching pressure of Year: 1995 of Month: 1
Dir Already exists: pressure
Fetching URL:  https://opendata.dwd.de/climate_environment/CDC/grids_germany/hourly/Project_TRY/pressure/PRED_199501.nc.gz
Fetched PRED_199501.nc
Decompressed PRED_199501.nc
Completed Writing PRED_199501.nc
Opened netCDF: PRED_199501.nc
Fetching humidity of Year: 1995 of Month: 1
Dir Already exists: humidity
Fetching URL:  https://opendata.dwd.de/climate_environment/CDC/grids_germany/hourly/Project_TRY/humidity/RH_199501.nc.gz
Fetched RH_199501.nc
Decompressed RH_199501.nc
Completed Writing RH_199501.nc
Opened netCDF: RH_199501.nc
Fetching vapor_pressure of Year: 1995 of Month: 1
Dir

KeyboardInterrupt: 